In [78]:
import pandas as pd
import json
import plotly.express as px
import plotly.graph_objects as go

df_position = pd.read_json("src/data/raw/ais_position.json")
df_static = pd.read_json("src/data/raw/ais_static.json")
df_types = pd.read_csv("src/data/raw/ais_ShipTypes.csv", sep=";")
df_status = pd.read_csv("src/data/raw/ais_NavigationStatus.csv", sep=";")

df_static['timestamp'] = pd.to_datetime(df_static['timestamp'].str.replace('UTC', ''), format='ISO8601').dt.floor('s')
df_position['timestamp'] = pd.to_datetime(df_position['timestamp'].str.replace('UTC', ''), format='ISO8601').dt.floor('s')
df_position.sort_values('MMSI')

,timestamp,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,UserID
26,2025-07-07 13:12:59+00:00,205133000,Z26 AVANTI,48.719662,-3.965290,293.6,7,0,0.0,0,205133000
31,2025-07-07 13:13:09+00:00,205133000,Z26 AVANTI,48.719662,-3.965287,97.0,7,0,0.0,0,205133000
58,2025-07-07 13:13:59+00:00,205133000,Z26 AVANTI,48.719665,-3.965293,50.7,7,0,0.1,0,205133000
83,2025-07-07 13:14:59+00:00,205133000,Z26 AVANTI,48.719663,-3.965293,133.5,7,0,0.1,0,205133000
116,2025-07-07 13:15:59+00:00,205133000,Z26 AVANTI,48.719663,-3.965290,137.7,7,0,0.0,0,205133000
...,...,...,...,...,...,...,...,...,...,...,...
8,2025-07-07 13:12:24+00:00,374496000,ROLLS I,48.288420,-4.740095,70.5,0,0,7.2,0,374496000
36,2025-07-07 13:13:24+00:00,374496000,ROLLS I,48.289080,-4.737187,71.0,0,0,7.4,0,374496000
94,2025-07-07 13:15:24+00:00,374496000,ROLLS I,48.290430,-4.731107,71.1,0,0,7.9,0,374496000
67,2025-07-07 13:14:24+00:00,374496000,ROLLS I,48.289757,-4.734200,72.6,0,0,7.7,0,374496000


In [79]:
# Keep LATEST MMSI static data
df_static_latest = df_static.sort_values("timestamp").drop_duplicates("MMSI", keep="last")
# Add the Ship Type string
df_static_latest = pd.merge(df_static_latest, df_types, on='Type', how='left')
# Calculate Lenght and Width
df_static_latest["Length"] = df_static_latest["Dimension"].apply(lambda x: x.get("A", 0) + x.get("B", 0))
df_static_latest["Width"] = df_static_latest["Dimension"].apply(lambda x: x.get("C", 0) + x.get("D", 0))

# Join Navigation Status to position
df_position_final = pd.merge(df_position, df_status, on='NavigationalStatus', how='left')

# final_df
final_df = pd.merge(df_position_final, df_static_latest, on='MMSI', how='left', suffixes = ('_position', '_static'))
final_df

# QUESTION --> EST CE QUE JE GARDE TOUTES LES POISTIONS ? SI OUI AJOUTER UN POSITIONSTATUS : 1 = Latest, 0 = Old

,timestamp_position,MMSI,ShipName,lat,lon,COG,NavigationalStatus,RateOfTurn,SOG,Spare,...,timestamp_static,Destination,Dimension,Eta,Type,Type Description,Category,SubCategory,Length,Width
0,2025-07-07 13:12:13+00:00,227007050,FRANCOIS ANDRE,48.737450,-4.009000,299.0,8,-128,5.6,0,...,2025-07-07 13:12:14+00:00,,"{'A': 24, 'B': 6, 'C': 6, 'D': 4}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",0.0,Not available (default),Not Used,NaN,30.0,10.0
1,2025-07-07 13:12:14+00:00,228381900,SAPEUR,48.369800,-4.513267,189.0,5,0,0.0,0,...,2025-07-07 13:16:01+00:00,FRENCH STATE ORDER,"{'A': 21, 'B': 53, 'C': 8, 'D': 9}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",52.0,Tug,Tug,NaN,74.0,17.0
2,2025-07-07 13:12:14+00:00,227806400,FRENCH WARSHIP EGL,48.367813,-4.517283,360.0,5,-128,0.0,0,...,2025-07-07 13:12:46+00:00,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",0.0,Not available (default),Not Used,NaN,0.0,0.0
3,2025-07-07 13:12:15+00:00,227008710,,48.381045,-4.486818,193.0,15,-128,0.1,0,...,2025-07-07 13:13:55+00:00,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",0.0,Not available (default),Not Used,NaN,0.0,0.0
4,2025-07-07 13:12:15+00:00,227574030,BINDY,48.369127,-4.515283,314.5,15,-128,7.3,0,...,2025-07-07 13:13:08+00:00,,"{'A': 13, 'B': 23, 'C': 6, 'D': 4}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",60.0,"Passenger, all ships of this type",Passenger,all ships of this type,36.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119,2025-07-07 13:16:08+00:00,227999999,FRENCH WARSHIP,48.369153,-4.514295,256.5,0,0,0.0,0,...,2025-07-07 13:12:38+00:00,,"{'A': 0, 'B': 0, 'C': 0, 'D': 0}","{'Day': 7, 'Hour': 7, 'Minute': 7, 'Month': 7}",35.0,Military ops,Military,NaN,0.0,0.0
120,2025-07-07 13:16:09+00:00,227006650,MISTRAL 7,48.381095,-4.484482,280.4,5,-128,0.0,0,...,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
121,2025-07-07 13:16:10+00:00,228429700,TSM BATZ,48.382048,-4.481558,293.0,5,0,0.0,0,...,2025-07-07 13:16:11+00:00,BREST,"{'A': 12, 'B': 12, 'C': 4, 'D': 5}","{'Day': 5, 'Hour': 20, 'Minute': 30, 'Month': 7}",52.0,Tug,Tug,NaN,24.0,9.0
122,2025-07-07 13:16:13+00:00,227007050,FRANCOIS ANDRE,48.741700,-4.013833,326.0,8,-128,2.6,0,...,2025-07-07 13:12:14+00:00,,"{'A': 24, 'B': 6, 'C': 6, 'D': 4}","{'Day': 0, 'Hour': 24, 'Minute': 60, 'Month': 0}",0.0,Not available (default),Not Used,NaN,30.0,10.0


In [80]:
final_df["custom_hover"] = (
    final_df["MMSI"].astype(str) + " - " + final_df["ShipName"] + "<br>" +
    "Category : " + final_df["Category"] + "<br>" +
    "Status : " + final_df["Status Description"] + "<br>" +
    "Destination : " + final_df["Destination"] + "<br>" +
    "Position at : " + final_df["timestamp_position"].astype(str)
)


fig = px.scatter_map(final_df, lat="lat", lon="lon", color="Status Description",
                     color_continuous_scale=px.colors.cyclical.IceFire
                     )


for key, group in final_df.groupby(['MMSI', 'UserID', 'ShipName', 'NavigationalStatus']):
    # Sort group if needed for order of lines
    group_sorted = group.sort_values(['timestamp_position']) # or by time if you have it
    print(group_sorted)

    fig.add_trace(go.Scattermap(
        lat=group_sorted['lat'],
        lon=group_sorted['lon'],
        mode='lines',
        hoverinfo = 'text',
        line=dict(color="white", width=1),
        hovertemplate=(
        "<b>%{customdata[0]} - %{customdata[1]}</b><br>" +
        "Category: %{customdata[2]}<br>" +
        "Status: %{customdata[3]}<br>" +
        "Destination: %{customdata[4]}<br>" +
        "Position at: %{customdata[5]}<br><extra></extra>"
        ),
        customdata= group_sorted[[
            'MMSI', 'ShipName', 'Category', 'Status Description', 'Destination', 'timestamp_position'
            ]].values,
    showlegend=False  # or True if you want legends for lines
    ))

fig.show()

           timestamp_position       MMSI    ShipName        lat       lon  \
26  2025-07-07 13:12:59+00:00  205133000  Z26 AVANTI  48.719662 -3.965290   
31  2025-07-07 13:13:09+00:00  205133000  Z26 AVANTI  48.719662 -3.965287   
58  2025-07-07 13:13:59+00:00  205133000  Z26 AVANTI  48.719665 -3.965293   
83  2025-07-07 13:14:59+00:00  205133000  Z26 AVANTI  48.719663 -3.965293   
116 2025-07-07 13:15:59+00:00  205133000  Z26 AVANTI  48.719663 -3.965290   

       COG  NavigationalStatus  RateOfTurn  SOG  Spare  ...     Destination  \
26   293.6                   7           0  0.0      0  ...  FISHINGGROUNDS   
31    97.0                   7           0  0.0      0  ...  FISHINGGROUNDS   
58    50.7                   7           0  0.1      0  ...  FISHINGGROUNDS   
83   133.5                   7           0  0.1      0  ...  FISHINGGROUNDS   
116  137.7                   7           0  0.0      0  ...  FISHINGGROUNDS   

                              Dimension  \
26   {'A': 28, 'B':

In [ ]:
px.line_map()